<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/Semi_UL_check_Cross_Light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics

f1_score_array = []
unlabel_size_array = []
train_data_size_array = []
total_train_array = []
validation_f1_score_array =[]
def Training_UL_RF(unlabel_size,Threshold):
    
    import pandas as pd
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

    train1 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data1.csv')
    train2 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data2.csv')
    train3 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data3.csv')
    train4 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data4.csv')
    train5 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data5.csv')
    train6 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data6.csv')
    train7 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data7.csv')
    train8 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data8.csv')
    train9 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data9.csv')
    train10 = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data10.csv')
    train_highKappa = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Train_data\train_data_highkappa.csv')

    #train1.head()
    train = train1
    train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
    for i in train_list:
      print(i)
      train = train.append(i)

    unlabel_size = unlabel_size
    Threshold=Threshold
    train.sort_values("Sentence", inplace = True)
    print(len(train))

    train = train.drop_duplicates(subset ="Sentence")

    train['Target'].unique()


    train['Target']=train['Target'].replace(['Others'],'Invalid')
    train['Target'].unique()



    #cleaning
    import nltk
    import re
    import string
    nltk.download('stopwords')
    nltk.download('wordnet')
    stopword=nltk.corpus.stopwords.words('english')
    from nltk.stem import WordNetLemmatizer
    wl= WordNetLemmatizer()

    def clean_text(text):
      text="".join([word.lower() for word in text if word not in string.punctuation])
      tokens = re.split('\W+',text)
      text = [wl.lemmatize(word) for word in tokens if word not in stopword]
      return text

    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf_vect = TfidfVectorizer(analyzer = clean_text)
    X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
    print(X_tfidf.shape)

    test = pd.read_csv(r'C:\Users\iia\Documents\Supriya\test_data.csv')
    test['Target']=test['Target'].replace(['Others'],'Invalid')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
    from nltk.corpus import stopwords
    words = stopwords.words('english')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
    t_p = tfidf_vect.transform(test['Sentence'])

    import numpy as np
    import statistics
    from sklearn.datasets import make_classification
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import make_pipeline
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import confusion_matrix
    from sklearn.base import BaseEstimator, ClassifierMixin
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import average_precision_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report



    X, y = make_classification(random_state=1)

    X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.20,random_state=42)

    classifier = lgb.LGBMClassifier()
    classifier.fit(X_train, Y_train)
    y_pred = classifier.predict(x_val)
    #print(np.unique(y_pred))
    #print(type(y_pred))

    Accuracy_score = accuracy_score(y_val,y_pred)
    test_pred = classifier.predict(t_p)
    acc_test = accuracy_score(test['Target'], test_pred)
    print('Accuracy test data', acc_test)
    classification_report_supervised = classification_report(test['Target'], test_pred, digits = 4)

    print(classification_report_supervised)
    matrix = confusion_matrix(y_val, y_pred)
    print(matrix)


    unlabel = pd.read_csv(r'C:\Users\iia\Documents\Supriya\Unlabeled_data.csv')
    #unlabel.head()

    del unlabel['Complete']
    del unlabel['Unnamed: 0']

    unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
    from nltk.corpus import stopwords
    words = stopwords.words('english')
    unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


    from textblob import TextBlob
    from textblob import Word
    nltk.download('wordnet')
    nltk.download('punkt')
    unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
    unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

    unlabel_1 = unlabel.loc[:unlabel_size]

    def index_reset(unlabel_2):
      unlabel_2.reset_index(inplace=True)
      del unlabel_2['index']
      #print(unlabel_2.head())
      return unlabel_2

    unlabel_1 = index_reset(unlabel_1)

    x_un1 = tfidf_vect.transform(unlabel_1['text'])

    x_un1.shape
    pred_unlabel_1 = classifier.predict_proba(x_un1)
    pred_unlabel_1

    x_un1.shape

    import numpy as np
    pos=[]
    large=[]
    ind = []
    i=0
    for j in pred_unlabel_1:
      if max(j)> Threshold:
        ind.append(np.argmax(j))
        large.append(max(j))
        pos.append(i)
      i+=1


    print(ind)
    print(large)
    print(pos)
    print(type(pos))
    print(len(ind))
    print(len(large))
    print(len(pos))

    unlabel_1 = unlabel_1.loc[pos,:]
    train_data_size = len(unlabel_1)
    print(len(unlabel_1))
    #unlabel_1.head()

    class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

    class_x_un1.shape


    class_pred_unlabel_1 = classifier.predict(class_x_un1)
    class_pred_unlabel_1


    unlabel_1['Target']=class_pred_unlabel_1
    #unlabel_1.head()
    
    train = train.rename(columns={'Sentence':'text'})
    frame_1 = [train,unlabel_1]
    train_1 = pd.concat(frame_1)
    
    print("#######3train len",len(train))
    print("UL ",len(unlabel_1))
    print("train_1",len(train_1))
    total_train = len(train_1)
    del train
    del unlabel_1
    del unlabel



    from sklearn.metrics import classification_report
    from sklearn import svm

    x_train_1 = tfidf_vect.transform(train_1['text'])
    x_train_1.shape
    X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1['Target'],test_size=0.25,random_state=2)
    classifier_1 = lgb.LGBMClassifier()
    classifier_1.fit(X_train, Y_train)
    y_pred = classifier_1.predict(x_val)
    Accuracy_score = accuracy_score(y_val, y_pred)

    print('Validation ------- Accuracy_score: %f' % Accuracy_score)
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    scores_f1 = cross_val_score(classifier_1,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    #avg_scores = statistics.mean(scores) 
    print("@@@@@@@cross validation scores ",scores_f1)
    
    scores_acc = cross_val_score(classifier_1,x_val ,y_val, scoring = 'accuracy', cv=cv, n_jobs=-1).mean()
    #avg_scores = statistics.mean(scores) 

    print("@@@@@@@cross validation scores ",scores_acc)
    scores_pre = cross_val_score(classifier_1,x_val ,y_val, scoring = 'precision', cv=cv, n_jobs=-1).mean()
    scores_recall = cross_val_score(classifier_1,x_val ,y_val, scoring = 'recall', cv=cv, n_jobs=-1).mean()
 


    test_pred = classifier_1.predict(t_p)
    acc_test = accuracy_score(test['Target'],test_pred)
    print('Test ------Semi Supervised Accuracy score ', acc_test)
    matrix = confusion_matrix( y_val,y_pred)
    print(matrix)
    print("---------validation--------------")
    print(classification_report(y_val,y_pred,digits=4))
    print("----------test---------------------")
    classification_report_semi_supervised = classification_report(test['Target'],test_pred, digits=4)
    f1_Score_metric = f1_score(test['Target'],test_pred,average='weighted')
    
    print(classification_report_semi_supervised)
    print(f1_Score_metric)


    # Write line to file

    with open(r'C:\Users\iia\Documents\Supriya\Results_3\SS_UL_Light.txt', 'a') as writefile:
      writefile.write("\n")
      writefile.write(" RF ")
      writefile.write("\n")
      writefile.write("Threshold = ")
      writefile.write(str(Threshold))
      writefile.write("\n")
      writefile.write("unlabel size  = ")
      writefile.write(str(unlabel_size))
      writefile.write("\n")
      writefile.write("train data size  = ")
      writefile.write(str(train_data_size))
      writefile.write("\n")
      writefile.write("total_train = ")  
      writefile.write(str(total_train))
      writefile.write("\n")  
      writefile.write("Supervised Resutls on Test Data")
      writefile.write("\n")
      writefile.write(classification_report_supervised)
      writefile.write("\n")
      writefile.write("Semi Supervised Resutls on Test Data")
      writefile.write("\n")
      writefile.write(classification_report_semi_supervised)
      writefile.write("Semi Supervised F1score on Test Data")
      writefile.write("\n")
      writefile.write(" Test f1 score test ")
      writefile.write(str(f1_Score_metric))
      writefile.write("\n")
      writefile.write("training accuracy ")  
      writefile.write(str(scores_acc))
      writefile.write("\n")   
      writefile.write("training precision ") 
      writefile.write(str(scores_pre))  
      writefile.write("\n")   
      writefile.write("training recall ") 
      writefile.write(str(scores_recall))
      writefile.write("\n")   
      writefile.write("training f1 score ")
      writefile.write(str(scores_f1))  
      writefile.write("\n") 


    print("Done")
    f1_score_array.append(f1_Score_metric)
    validation_f1_score_array.append(scores_f1)
    unlabel_size_array.append(unlabel_size)
    train_data_size_array.append(train_data_size)
    total_train_array.append(total_train)
    
    return f1_score_array,unlabel_size_array,train_data_size_array,validation_f1_score_array,total_train_array

In [ ]:
unlabel_size_list = range(1000,500000,5000)
unlabel_size_list[5]

In [ ]:
len(unlabel_size_list)

In [ ]:



Threshold = 0.99
iteration = 0 

for size in unlabel_size_list:
    print("iteration",iteration)
    print("size of the data got into ", size )
    op_f1_score,op_unlabel_size,op_train_data_size,op_val_f1_array,op_total_train_array = Training_UL_RF(size,Threshold)
    iteration = iteration + 1
    
with open(r'C:\Users\iia\Documents\Supriya\Results_3\SS_UL_Light_variables.txt', 'a') as writefile:
    writefile.write("\n")
    writefile.write(" RF ")
    writefile.write("\n")
    writefile.write("\n")
    writefile.write("op_f1_score")
    writefile.write(str(op_f1_score))
    writefile.write("\n")
    writefile.write("op_unlabel_size")
    writefile.write("\n")
    writefile.write(str(op_unlabel_size))
    writefile.write("\n")
    writefile.write("op_train_data_size")
    writefile.write("\n")
    writefile.write(str(op_train_data_size))
    writefile.write("\n")
    writefile.write("op_val_f1_array")
    writefile.write(str(op_val_f1_array))
    writefile.write("\n")
    writefile.write("op_total_train_array = ")
    writefile.write(str(op_total_train_array))
